<a href="https://colab.research.google.com/github/tifftknguyen/Final-Project-456/blob/tiff/final-project-456.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiffany Nguyen
# Giselle Hernandez
# CECS 456 Sec 01 - Machine Learning
# Professor Wenlu Zhang
# Final Project

In [10]:
! pip install -q kaggle

In [11]:
import kagglehub

In [14]:
path = kagglehub.dataset_download("prasunroy/natural-images")

Using Colab cache for faster access to the 'natural-images' dataset.


In [15]:
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/natural-images
